In [1]:
# https://taxfoundation.org/data/all/state/state-income-tax-rates-2024/
import pandas as pd
tax_bracket_data = pd.read_excel("State-Individual-Income-Tax-Rates-and-Brackets-2015-2024_Tax_Foundation.xlsx", sheet_name=None)
state_dct ={
    "Ala.":"AL",
    "Alaska":"AK",
    "Ariz.":"AZ",
    "Ark.":"AR",
    "Calif.":"CA",
    "Colo.":"CO",
    "Conn.":"CT",
    "Del.":"DE",
    "Fla.":"FL",
    "Ga.":"GA",
    "Hawaii":"HI",
    "Idaho":"ID",
    "Ill.":"IL",
    "Ind.":"IN",
    "Iowa":"IA",
    "Kans.":"KS",
    "Ky.":"KY",
    "La.":"LA",
    "Maine":"ME",
    "Md.":"MD",
    "Mass.":"MA",
    "Mich.":"MI",
    "Minn.":"MN",
    "Miss.":"MS",
    "Mo.":"MO",
    "Mont.":"MT",
    "Nebr.":"NE",
    "Nev.":"NV",
    "N.H.":"NH",
    "N.J.":"NJ",
    "N.M.":"NM",
    "N.Y.":"NY",
    "N.C.":"NC",
    "N.D.":"ND",
    "Ohio":"OH",
    "Okla.":"OK",
    "Ore.":"OR",
    "Pa.":"PA",
    "R.I.":"RI",
    "S.C.":"SC",
    "S.D.":"SD",
    "Tenn.":"TN",
    "Texas":"TX",
    "Utah":"UT",
    "Vt.":"VT",
    "Va.":"VA",
    "Wash.":"WA",
    "W.Va.":"WV",
    "Wis.":"WI",
    "Wyo.":"WY",
    "D.C.":"DC"
    }

state_dct2 = {"Alabama":"AL",
              "Alaska":"AK", 
              "Arizona": "AZ",
                "Arkansas":"AR",
                "California":"CA",
                "Colorado":"CO",
                "Connecticut":"CT",
                "Delaware":"DE",
                "Florida":"FL",
                "Georgia":"GA",
                "Hawaii":"HI",
                "Idaho":"ID",
                "Illinois":"IL",
                "Indiana":"IN",
                "Iowa":"IA",
                "Kansas":"KS",
                "Kentucky":"KY",
                "Louisiana":"LA",
                "Maine":"ME",
                "Maryland":"MD",
                "Massachusetts":"MA",
                "Michigan":"MI",
                "Minnesota":"MN",
                "Mississippi":"MS",
                "Missouri":"MO",
                "Montana":"MT",
                "Nebraska":"NE",
                "Nevada":"NV",
                "New Hampshire":"NH",
                "New Jersey":"NJ",
                "New Mexico":"NM",
                "New York":"NY",
                "North Carolina":"NC",
                "North Dakota":"ND",
                "Ohio":"OH",
                "Oklahoma":"OK",
                "Oregon":"OR",
                "Pennsylvania":"PA",
                "Rhode Island":"RI",
                "South Carolina":"SC",
                "South Dakota":"SD",
                "Tennessee":"TN",
                "Texas":"TX",
                "Utah":"UT",
                "Vermont":"VT",
                "Virginia":"VA",
                "Washington":"WA",
                "West Virginia":"WV",
                "Wisconsin":"WI",
                "Wyoming":"WY",
                "District of Columbia":"DC"
                }


In [2]:
min_max_dct = {"Min":{},
               "Max":{}}
single_married = ["Single Filer","Married Filing Jointly"]
min_max_dct = {min_max: {sm: {year: {} for year in tax_bracket_data.keys()} for sm in single_married} for min_max in ("Min","Max")}


for year in tax_bracket_data.keys():
    tax_bracket_data[year] = tax_bracket_data[year].iloc[1:]
    tax_bracket_data[year]["State"] = tax_bracket_data[year]["Unnamed: 0"].map(state_dct).fillna(method='ffill')
    for sm in single_married:
        tax_bracket_data[year][sm] = pd.to_numeric(tax_bracket_data[year][sm], errors =  "coerce")
       
        for min_max in ("min","max"):
            min_max_data = tax_bracket_data[year][["State", sm]]
            min_max_dct[min_max.title()][sm][year] = getattr(min_max_data.groupby("State"), min_max)()[sm]


/tmp/ipykernel_68181/2502699505.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tax_bracket_data[year]["State"] = tax_bracket_data[year]["Unnamed: 0"].map(state_dct).fillna(method='ffill')
/tmp/ipykernel_68181/2502699505.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tax_bracket_data[year]["State"] = tax_bracket_data[year]["Unnamed: 0"].map(state_dct).fillna(method='ffill')
/tmp/ipykernel_68181/2502699505.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tax_bracket_data[year]["State"] = tax_bracket_data[year]["Unnamed: 0"].map(state_dct).fillna(method='ffill')
/tmp/ipykernel_68181/2502699505.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.

In [3]:
min_max_dfs = {}
for sm in single_married:
    min_max_dfs[sm] = {}
    for min_max in ("min", "max"):
        min_max_dfs[sm][min_max] = pd.DataFrame(min_max_dct[min_max.title()][sm])
    min_max_dfs[sm] = pd.concat(min_max_dfs[sm].values(), keys = min_max_dfs[sm].keys()).reset_index()
    min_max_dfs[sm] = min_max_dfs[sm].rename(columns = {"level_0":"minmax"}).set_index(["State", "minmax"]).sort_index().sort_index(axis=1)
    min_max_dfs[sm] = min_max_dfs[sm].reset_index().melt(id_vars = ["State", "minmax"], var_name = "Year", value_name = "Income Tax Rate")
    


In [4]:
from homebrewedFunctions.functions import *
import plotly.express as px
regions_df = pd.read_csv("USCensusRegions.csv")
dfs = {}
# for min_max in ("Min", "Max"):
for sm in single_married:
    dfs[sm] = min_max_dfs[sm].pivot(index = ["State", "Year"], columns = "minmax", values = "Income Tax Rate").dropna().reset_index()
    dfs[sm]["Year"] = pd.to_datetime(dfs[sm]["Year"], format = "%Y")
    dfs[sm].sort_values(by = "Year", inplace = True)
    dfs[sm]["Year"] = dfs[sm]["Year"].dt.year.astype(str)
    dfs[sm].set_index(["State", "Year"], inplace = True)
    dfs[sm].sort_index()
figs = aggregated_line_dropdown(dfs, regions_df, f"TaxBrackets")
figs.write_html(f"outputs/State government finances/TaxBrackets.html")


# figs = {}
#     for state in df["State"].unique():
#         fig = px.line(df[df["State"]] == state],
#         x="Year", 
#         y="Income Tax Rate",
#         color="State",
#         facet_col="minmax")



In [5]:
CITR = pd.read_csv("2024 State Corporate Income Tax Rates  Brackets.csv")
CITR["State"] = CITR["State"].map(state_dct2)
CITR["Rates"] = CITR["Rates"].str.replace("%", "").astype(float).div(100)
min_CITR = CITR.groupby("State")["Rates"].min()
max_CITR = CITR.groupby("State")["Rates"].max()
# CITR = pd.concat([min_CITR, max_CITR], keys = ["Min", "Max"]).reset_index()
# CITR = CITR.rename(columns = {"level_0":"minmax"}).set_index(["State", "minmax"]).sort_index().sort_index(axis=1)
# CITR = CITR.reset_index().melt(id_vars = ["State", "minmax"], var_name = "Year", value_name = "Corporate Income Tax Rate")
CITR = pd.DataFrame(max_CITR)#, columns = ["Corporate Income Tax Rate"]).reset_index()
CITR=CITR.reset_index().sort_values(by="Rates", ascending = False)

In [11]:
fig = px.bar(CITR, x = "State", y = "Rates")
fig.update_layout(title = "2024 State Corporate Income Tax Rates<br>(Maximum Rate for States with Non-Unifrom Schedule)")
fig.write_html(f"outputs/State government finances/2024 State Corporate Income Tax Rates .html")